## Create the Test Data for four scenerios

In [1]:
import os
os.chdir('../..')
from trips.config import *
from trips.weibull_trips import main
import subprocess

In [ ]:
scene_src_probabilities = {
    "High Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "Low Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "NS-Traffic Scenerio": [0.45, 0.05, 0.45, 0.05],
    "EW-Traffic Scenerio": [0.05, 0.45, 0.05, 0.45],
}

scene_number_cars = {
    "High Traffic Scenerio": 1000,
    "Low Traffic Scenerio": 150,
    "NS-Traffic Scenerio": 500,
    "EW-Traffic Scenerio": 500,
}


if not os.path.exists("test"):
    os.makedirs("test")
    
for scene in scene_src_probabilities:
    main(
        src_nodes,
        dst_nodes,
        scene_src_probabilities[scene],
        turn_prob,
        emergency_probs,
        shape,
        scene_number_cars[scene],
        total_timesteps,
    )
    subprocess.run(
        [
            "duarouter",
            "-n",
            "./network_details/intersection.net.xml",  
            "-t",
            "trips.trips.xml",  
            "-o",
            f"test/intersection_{scene}.rou.xml",  
        ]
    )
os.remove("trips.trips.xml")


route_files = [f"test/intersection_{scene}.rou.xml" for scene in scene_src_probabilities]
        

 Use 'route-files' instead.
 Use 'route-files' instead.


Success.up to time step: 5243.00
Success.up to time step: 5231.00


 Use 'route-files' instead.
 Use 'route-files' instead.


Success.up to time step: 5309.00
Success.up to time step: 5360.00


## Test round robin algorithm on the scenerios

In [3]:
from environment.environment import MultiRouteSumoEnvironment

In [ ]:
for i in range(4):
    env = MultiRouteSumoEnvironment(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/test/{i}/RoundRobin',
        min_green=25,
        yellow_time=5,
        delta_time=30,
        use_gui=False,
        num_seconds=5400
    )

    obs, info = env.reset()
    done = False

    action_index = 0  
    while not done:
        action = action_index % 4
        
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        action_index += 1

    obs, info = env.reset()
    env.close()

## Test trained DQN Policy on the scenerios

In [ ]:
from stable_baselines3 import DQN

model = DQN.load("agents/dqn_multi_scene")

for i in range(4):
    env = MultiRouteSumoEnvironment(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/test/{i}/DQN',
        min_green=5,
        yellow_time=5,
        delta_time=10,
        use_gui=False,
        num_seconds=5400
    )
    
    obs, info = env.reset()
    done = False

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

    obs, info = env.reset()
    env.close()

## Test the trained A2C algorithm on the scenerios

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv

model = A2C.load('agents/a2c_multi_scene')

for i in range(4):
    def make_env():
        return MultiRouteSumoEnvironment(
            net_file="network_details/intersection.net.xml",
            route_files=route_files,
            out_csv_name=f'outputs/test/{i}/A2C',
            min_green=5,
            yellow_time=5,
            delta_time=10,
            use_gui=False,
            num_seconds=5400
        )
        
    test_env = DummyVecEnv([make_env])

    obs = test_env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = test_env.step(action)
